### Prerequisites: Simplex-Algorithm-Linear-Programming

---

### Context

Let's imagine a farmer who needs people to pick leaves and pumpkins in a large field. He'll hire a lot of people, but I don't know how many yet.

He'll recruit students from ten different schools to help him pick the leaves and pumpkins. Depending on the school, each student will claim a different salary. And the quality of the leaf and pumpkin collection will vary from school to school.

Based on the data below, we want to determine the percentage $ x_1 \% $ of people who will come from School 1, $ x_2 \% $ from School 2, and so on, to form the team.

| School No. | 1  | 2  | 3  | 4  | 5  | 6  | 7  | 8  | 9  | 10 |
|------------|----|----|----|----|----|----|----|----|----|----|
| **Leaves** | 4.9| 2.5| 3.7| 4.2| 3.1| 4.6| 3.3| 4.7| 2.6| 4.5|
| **Pumpkins** | 3.8| 4.0| 3.6| 4.1| 3.9| 4.1| 4.3| 3.9| 3.1| 3.1|
| **€/hour** | 17 | 24 | 15 | 16 | 20 | 28 | 23 | 18 | 12 | 17 |

It is crucial for the farmer that the garden is thoroughly cleaned, so he requires a global quality of at least 4 for both leaf and pumpkin collection.

Given these conditions, he wants to minimize the total cost of the operation.


### Mathematical Reformulation and Standard Form:

The problem can be reformulated as follows:

#### Objective:
Minimize $ 17x_1 + 24x_2 + 15x_3 + \cdots + 17x_{10} $

#### Subject to the constraints:
$$
\begin{aligned}
4.9x_1 + 2.5x_2 + \cdots + 4.5x_{10} &\geq 4 \\
3.8x_1 + 4x_2 + \cdots + 3.1x_{10} &\geq 4 \\
x_1 + x_2 + x_3 + \cdots + x_{10} &= 1 \\
x_i &\geq 0 \quad \text{for all } i = 1, 2, \ldots, 10
\end{aligned}
$$

#### Standard Form:
The problem is written in standard form using slack variables $ x_{11}$ and $ x_{12} $:

$$
\begin{aligned}
4.9x_1 + 2.5x_2 + \cdots + 4.5x_{10} - x_{11} &= 4 \\
3.8x_1 + 4x_2 + \cdots + 3.1x_{10} - x_{12} &= 4 \\
x_1 + x_2 + x_3 + \cdots + x_{10} &= 1 \\
x_i &\geq 0 \quad \text{for all } i = 1, 2, \ldots, 12
\end{aligned}
$$


### First Solution Method: Simplex Algorithm

We can solve this problem using the Simplex Algorithm. For more details, see Simplex-Algorithm-Linear-Programming on GitHub.

Using this method, we obtain the following results:
- $ x_4 = 0.9 $
- $ x_9 = 0.1 $
- $ x_{11} = 0.04 $ (indicating that the pumpkin efficiency constraint is the most restrictive, so it is active)
- For all $ i \notin \{4, 9\} $, $ x_i = 0 $

The minimum cost is $ 15.6 \, € / \text{hour/worker} $.

This means that the chosen team will consist of 90% of people from School 4 and 10% of people from School 9.

### Second Solution Method: Column Generation

#### Why Not Rely Solely on the First Method?

The Simplex Algorithm requires recalculating the reduced costs of all variables at each iteration, which is not ideal in terms of computational efficiency.

#### Why Is This Method Called "Column Generation"?

Refer to the matrix formulation of the problem and the matrix computations in the Simplex-Algorithm-Linear-Programming repository.

#### What Is Column Generation?

The idea is to avoid recalculating the reduced costs of all variables in every iteration. We often have an intuition about the columns that will contribute to optimality.

In the leaf and pumpkin collection example, we might intuit that the optimal basis lies within the set $ S = \{x_4, x_5, x_9, x_{11}, x_{12}\} $. Essentially, this is a "pre-selection" of the schools that are potentially optimal. Additionally, the set $ S $ includes one or more artificial variables, which are suspected to correspond to constraints that may not be active. In this case, we are unsure which constraint will be active, so we include both $ x_{11} $ and $ x_{12} $ to account for this uncertainty.

Next, we apply the Simplex Algorithm to the pre-selected set, considering only Schools 4, 5, and 9, as if the others did not exist. Applying the algorithm to a smaller number of schools obviously reduces the computational burden.

In other words, we solve the following problem:

#### Objective:
Minimize $ 16x_4 + 20x_5 + 12x_9 $

#### Subject to the constraints:
$$
\begin{aligned}
4.2x_4 + 3.1x_5 + 2.6x_9 - x_{11} &= 4 \\
4.1x_4 + 3.9x_5 + 3.1x_9 - x_{12} &= 4 \\
x_4 + x_5 + x_9 &= 1 \\
x_i &\in [0, 1] \quad \text{for all } i
\end{aligned}
$$

Thus, the Simplex Algorithm is applied, but we no longer need to compute the reduced costs for the variables $ x_i $ corresponding to the non-selected schools.

The results obtained are:
- $ x_4 = 0.9 $
- $ x_9 = 0.1 $
- $ x_{11} = 0.04 $
- $ x_5 = x_{12} = 0 $

Finally, it is necessary to verify that the reduced costs of all other schools are positive (note that they should be positive because we are minimizing, which is contrary to what is typically discussed in Simplex-Algorithm-Linear-Programming).


In [7]:
from sympy import Matrix, symbols, simplify
x1, x2, x3, x5, x6, x7, x8, x10, x12 = symbols('x1 x2 x3 x5 x6 x7 x8 x10 x12')
expression = Matrix([[4.2, 2.6, -1],
            [4.1, 3.1, 0],
            [1, 1, 0]]).inv() * (Matrix([4, 4, 1]) - Matrix([[4.9, 2.5, 3.7, 3.1, 4.6, 3.3, 4.7, 4.5, 0],
             [3.8, 4.0, 3.6, 3.9, 4.1, 4.3, 3.9, 3.1, -1],
             [1, 1, 1, 1, 1, 1, 1, 1, 0]]) * Matrix([x1, x2, x3,x5, x6, x7,x8, x10, x12]))

x_b = simplify(expression)
x_b


Matrix([
[                                                                                    -0.7*x1 - 4.44089209850063e-16*x10 + 1.0*x12 - 0.9*x2 - 0.5*x3 - 0.8*x5 - 1.0*x6 - 1.2*x7 - 0.8*x8 + 0.9],
[-0.300000000000001*x1 - 1.0*x10 - 1.0*x12 - 0.100000000000001*x2 - 0.5*x3 - 0.200000000000001*x5 - 8.88178419700125e-16*x6 + 0.199999999999999*x7 - 0.200000000000001*x8 + 0.100000000000001],
[                                         1.18*x1 + 1.9*x10 + 1.6*x12 - 1.54*x2 + 0.299999999999999*x3 - 0.78*x5 + 0.399999999999999*x6 - 1.22*x7 + 0.819999999999999*x8 + 0.0400000000000005]])

In [8]:
sum = 17*x1 + 24*x2 + 15*x3 + 16*x_b[0] + 20*x5 + 28*x6 + 23*x7 + 18*x8 + 12*x_b[1] + 17*x10
objective = simplify(sum)
objective


2.19999999999999*x1 + 4.99999999999999*x10 + 4.0*x12 + 8.39999999999999*x2 + 0.999999999999988*x3 + 4.79999999999999*x5 + 12.0*x6 + 6.19999999999999*x7 + 2.79999999999999*x8 + 15.6

We can see that all the reduced costs are positive, which means that $(x_4=0.9,x_9=0.1)$ is indeed optimal!

#### And if we hadn't had the right intuition about S, what could we do?

If some of the reduced costs are positive, add the variable corresponding to the highest reduced cost (in minimization) to the set S. And reapply the simplex algorithm to this set.

## Conclusion

1) Always start by putting into standard form \
2) Use your intuition to choose a subset S, which also contains deviation variables. \
3) Apply the simplex algorithm to S until an optimum is found. \
4) Check whether this optimum is also an optimum for the initial problem with reduced costs. If not, add the variable with the highest (minimization) or lowest (maximization) reduced cost, then start again. \
5) If the initial intuition is right, the column generation method allows you to reach the optimum with far fewer calculations than by applying the simplex to all variables. \